In [63]:
from typing import Sequence, TypedDict, Annotated
from langgraph.graph import START, StateGraph, END
from langchain_ollama import ChatOllama
from langchain_core.messages import BaseMessage, ToolMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

In [64]:
class AgentState(TypedDict):
    messages : Annotated[Sequence[BaseMessage], add_messages]

In [65]:
@tool
def add(a : int, b: int):
    """This addition function takes 2 integers as input and returns their sum as output"""
    return a + b

@tool
def subtract(a : int, b: int):
    """This subtract function takes 2 integers as input and returns their difference as output"""
    return a - b

@tool
def multiply(a : int, b: int):
    """This multiply function takes 2 integers as input and returns their product as output"""
    return a * b

In [66]:
tools = [add, subtract, multiply]
model = ChatOllama(model="llama3.2").bind_tools(tools)

In [67]:
def model_call(state : AgentState) -> AgentState:
    # Explicitly telling the model NOT to talk when calling tools
    system_prompt = SystemMessage(
        content = "You are a helpful assistant. "
                  "If you need to use a tool, output the tool call only without preamble."
    )
    response = model.invoke([system_prompt] + state['messages'])
    return {"messages" : [response]}

In [68]:
def should_continue(state : AgentState) -> str:
    messages = state['messages']
    last_message = messages[-1]
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"

In [69]:
graph = StateGraph(AgentState)
graph.add_node("my_agent", model_call)
graph.add_edge(START, "my_agent")
tool_node = ToolNode(tools = tools)
graph.add_node("tools", tool_node)
graph.add_conditional_edges(
    "my_agent",
    should_continue,
    {
        "end" : END,
        "continue" : "tools"
    }
)
graph.add_edge("tools", "my_agent")
app = graph.compile()

In [70]:
def print_stream(stream):
    for s in stream:
        # 'values' mode returns the full state at each step
        message = s["messages"][-1]
        message.pretty_print()

In [71]:
inputs = {"messages": [("user", "Add 40 + 12. Call that value x. Multiply x by 6 and show as final output. Also tell me a joke please.")]}
print_stream(app.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Add 40 + 12. Call that value x. Multiply x by 6 and show as final output. Also tell me a joke please.
================================== Ai Message ==================================
Tool Calls:
  multiply (bf079c4f-0623-4928-8a97-e72f661690cb)
 Call ID: bf079c4f-0623-4928-8a97-e72f661690cb
  Args:
    a: 52
    b: 6
================================= Tool Message =================================
Name: multiply

312
================================== Ai Message ==================================

Why couldn't the bicycle stand up by itself?

Because it was two-tired! (get it?)
